In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

import gm2
import trfp
import plotting_functions as plt2
import analysis_helper as helper
import helper_function_candidates as helper_old

# Load data, calculate vtms

In [ ]:
%%time
fp_interp_df = helper.root_to_pandas(range(3959,3995), prefix='v9_20_00/FieldPlainRootOutput_', tr_run=False)
tr_interp_df_1 = helper.root_to_pandas([3956], prefix='v9_20_00/FieldPlainRootOutput_', tr_run=True)
tr_interp_df_2 = helper.root_to_pandas([3997], prefix='v9_20_00/FieldPlainRootOutput_', tr_run=True)

In [ ]:
%%time
fp_moment_df = helper.calc_moment_df(fp_interp_df)
tr_moment_df_1 = helper.calc_moment_df(tr_interp_df_1)
tr_moment_df_2 = helper.calc_moment_df(tr_interp_df_2)

In [ ]:
%%time
tr_corr_df_1 = helper_old.trolley_footprint_replacement(tr_moment_df_1)
tr_corr_df_2 = helper_old.trolley_footprint_replacement(tr_moment_df_2)

In [ ]:
%%time
tr_baseline_1, fp_baseline_1, baseline_time_1, summed_azimuth_1, summed_pts_1 = helper_old.trolley_run_station_average(tr_corr_df_1)
tr_baseline_2, fp_baseline_2, baseline_time_2, summed_azimuth_2, summed_pts_2 = helper_old.trolley_run_station_average(tr_corr_df_2)

In [ ]:
%%time
vtm_df = helper.vtm_calc(fp_moment_df,
                         baseline_time_1, baseline_time_2,
                         tr_baseline_1, tr_baseline_2,
                         fp_baseline_1, fp_baseline_2)

# Time and azimuthal averaging

## Read in ctags and $\omega_a$ subrun status for time averaging

In [ ]:
### Read in ctag info

subrun_df = pd.read_hdf('60hr_subrun.h5', key='ctag')
usable_subruns_df = subrun_df[subrun_df['ok']==True].copy()

### Bin by subrun

intervals = []
interval_centers = []
interval_range = []
for ii in range(usable_subruns_df['end_gps'].values.size):
    intervals.append(pd.Interval(usable_subruns_df['start_gps'].values[ii],
                                 usable_subruns_df['end_gps'].values[ii],
                                 closed='both'
                                )
                    )
    interval_centers.append((usable_subruns_df['start_gps'].values[ii] + usable_subruns_df['end_gps'].values[ii])/2)
    interval_range.append(usable_subruns_df['end_gps'].values[ii] - usable_subruns_df['start_gps'].values[ii])
    
intervals = pd.IntervalIndex(intervals)
interval_centers = np.array(interval_centers)
interval_range = np.array(interval_range)

vtm_bin_df = vtm_df.groupby(pd.cut(vtm_df.index, intervals)).mean()
vtm_bin_df['bin_range'] = interval_range
vtm_bin_df['start_gps'] = usable_subruns_df['start_gps'].values

vtm_bin_df = vtm_bin_df.merge(usable_subruns_df[['start_gps', 'ctags']])
vtm_bin_df.index = interval_centers

## Time average, weighted by ctags in "ok" subruns

In [ ]:
stms = ['st'+str(st)+',m'+str(m+1) for st in range(72) for m in range(6)]
time_avg_series = vtm_bin_df[stms].multiply(vtm_bin_df['ctags'], axis='index').sum()/vtm_bin_df['ctags'].sum()

## Naive azimuthal average, weighted by fixed probe station extent

In [ ]:
azi_avg_series = pd.Series(index = ['m' + str(m) for m in np.arange(6)+1])

for m in range(5):
    weight = summed_azimuth_1[:, m] + summed_azimuth_2[:, m]
    
    total_weight = np.nansum(weight)
    stm_list = ['st'+str(st)+',m'+str(m+1) for st in np.arange(72)]
    azi_avg_series['m'+str(m+1)] = time_avg_series[stm_list].multiply(weight).sum()/total_weight

# Legacy code/studies

## Alan studies

In [ ]:
import allantools

fig, ax = plt.subplots(1,1)
for st in range(7):
    stm = 'st'+str(st)+',m1'
    tau, adev, _, _ = allantools.mdev(fp_moment_df[stm].values/61.79, data_type='freq', taus='decade')
    plt.loglog(tau, adev, '.', label=stm)
plt.legend()
fig.set_size_inches(12,8)
fig.tight_layout()

## Sync Offsets

In [ ]:
sync_offsets, delta_t = helper.sync_offset_calc(tr_corr_df_1, tr_corr_df_2)

def gaussian(x, A, x0, sigma): return A*np.exp(-(x-x0)**2/2./sigma**2)

fig, axs = plt.subplots(2,3)
for i in range(2):
    for j in range(3):
        st = 3*i+j
        if st == 5: continue
            
        plt.sca(axs[i,j])
        hist, bins, _ = plt.hist(sync_offsets[:,st], bins=50)
        low, high = axs[i,j].get_xlim()
                
        bins = bins[0:-1] + 0.5*(bins[1]-bins[0])
        coeffs, _ = curve_fit(gaussian, bins, hist, p0=[1., 0., 10.])
        fit = gaussian(np.arange(low, high, 0.1), coeffs[0], coeffs[1], coeffs[2])
        plt.plot(np.arange(low,high,0.1), fit, label=r'$\omega_0$ = '+str(np.round(coeffs[1],1))+'\n$\sigma$ = '+str(np.round(coeffs[2],1)))
        plt.legend(loc=1)
        plt2.plt_set_labels(axs[i,j], 'sync offset (Hz)', '', 'm '+str(st+1))
        
        if st == 0: plt.xlim(-100,100)
        else: plt.xlim(-50,50)


fig.set_size_inches(12,8)
fig.tight_layout()

## Extended trolley averages

In [ ]:
# calculate extended trolley averages

print np.sum(tr_baseline_1*summed_azimuth_1, axis=0)/360

print '\n'

print np.sum(tr_baseline_2*summed_azimuth_2, axis=0)/360

## Legacy time/azimuthal averaging

In [ ]:
### Bin into the agreed upon bins

bins = np.arange(1524384055, 1524641055, 1000)-500  # bin edges
bin_centers = np.arange(1524384055, 1524640055, 1000)

vtm_bin_df = vtm_df.groupby(pd.cut(vtm_df.index, bins)).mean()
vtm_bin_df.index = bin_centers

test_df = vtm_bin_df.copy()

azi_avg_df = pd.DataFrame(np.zeros((test_df.shape[0],6)),
                         index = test_df.index,
                         columns = ['m' + str(m) for m in np.arange(6)+1])

for m in range(5):
    weight = summed_azimuth_1[:, m] + summed_azimuth_2[:, m]
    
    # de-weight station 5 (split into 4 and 6 by distance to each)
    weight[4] += weight[5] * (trfp.STATION_BARCODE_PHI[6]-trfp.STATION_BARCODE_PHI[5])/(trfp.STATION_BARCODE_PHI[6]-trfp.STATION_BARCODE_PHI[4])
    weight[6] += weight[5] * (trfp.STATION_BARCODE_PHI[5]-trfp.STATION_BARCODE_PHI[4])/(trfp.STATION_BARCODE_PHI[6]-trfp.STATION_BARCODE_PHI[4])
    weight[5] = 0
    
    total_weight = np.nansum(weight)
    stm_list = ['st'+str(st)+',m'+str(m+1) for st in np.arange(72)]
    azi_avg_df['m'+str(m+1)] = test_df[stm_list].multiply(weight).sum(axis=1)/total_weight
    
print_df = azi_avg_df[['m1','m2','m3','m5']].copy()/61.79
print_df['m1_err'] = 0.116
print_df['m2_err'] = 0.06
print_df['m3_err'] = 0.06
print_df['m5_err'] = 0.1

print_df = print_df.sort_index(axis='columns')
# print_df.head()
# print_df.to_csv('purcell_60hr_7-31.csv')

In [ ]:
# print all stations

fig, axs = plt.subplots(24,3)

for i in range(24):
    for j in range(3):
        plt.sca(axs[i,j])
        st = i*3 + j
        plt.plot(vtm_bin_df.index.values, vtm_bin_df['st'+str(st)+',m5']/61.79, '.', markersize=1, color='navy')
        plt2.plt_set_labels(axs[i,j], '', 'NS (ppm)', 'st '+str(st))
        plt2.plt_unix_time_to_CST(axs[i,j])

fig.set_size_inches(16, 80)
fig.tight_layout()

## Compare with Rachel

In [ ]:
rachel_m1 = np.loadtxt('rachel_m1_st135.txt')

fig, axs = plt.subplots(24,3)

for i in range(24):
    for j in range(3):
        plt.sca(axs[i,j])
        st = i*3 + j
#         if st == 71: continue
        plt.plot(vtm_bin_df.index.values, vtm_bin_df['st'+str(st)+',m1']/61.79 - rachel_m1[:,st], '.', markersize=1, color='navy')
        plt2.plt_set_labels(axs[i,j], '', 'Dipole (ppm)', 'st '+str(st))
        plt2.plt_unix_time_to_CST(axs[i,j])

fig.set_size_inches(16, 80)
fig.tight_layout()